## Forward 알아보기

In [5]:
import torch
import torch.nn as nn


#linear layer 선언
linear_model = nn.Linear(1, 1) #인스턴스화 해서 함수인 것처럼
linear_model(torch.tensor([50.0]))

tensor([33.3331], grad_fn=<AddBackward0>)

### model의 weight, bias 출력

In [6]:
linear_model.weight

Parameter containing:
tensor([[0.6667]], requires_grad=True)

In [7]:
linear_model.bias

Parameter containing:
tensor([-0.0014], requires_grad=True)

### 모듈 호출

In [8]:
x = torch.ones(1)
linear_model(x)

tensor([0.6653], grad_fn=<AddBackward0>)

## 배치 최적화

- 배치 수행하는 이유 : 연산량을 충분히 크게 만들어서 컴퓨팅 자원 최대한 활용하기 위함

In [9]:
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9]

# 임의로 차원 추가
t_c = torch.tensor(t_c).unsqueeze(1)
t_u = torch.tensor(t_u).unsqueeze(1)

print(t_u.shape)

torch.Size([6, 1])


In [10]:
linear_model = nn.Linear(1, 1)
optimizer = torch.optim.SGD(
    linear_model.parameters(), lr = 1e-2
)

In [12]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[0.1997]], requires_grad=True),
 Parameter containing:
 tensor([0.1640], requires_grad=True)]

### Training loop 선언

In [13]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    for epoch in range(1, n_epochs+1):
        t_p_train = model(t_u_train) #predictions
        loss_train = loss_fn(t_p_train, t_c_train) #loss calculation
        
        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        
        if epoch == 1 or epoch % 1000 == 0:
            print(f'Epoch {epoch}, Training loss {loss_train.item():.4f}, Validation loss {loss_val.item():.4f}')
        
        

In [17]:
t_c_train = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0, 8.0]).unsqueeze(1)
t_c_val = torch.tensor([6.0, 13.0, 21.0]).unsqueeze(1)

t_un_train = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9]).unsqueeze(1)
t_un_val = torch.tensor([48.4, 60.4, 68.4]).unsqueeze(1)



linear_model = nn.Linear(1, 1)
optimizer = torch.optim.SGD(linear_model.parameters(), lr = 1e-3)

training_loop(
    n_epochs=3000,
    optimizer = optimizer,
    model = linear_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val,
    t_c_train = t_c_train,
    t_c_val = t_c_val
)

print()
print(linear_model.weight, linear_model.bias)

Epoch 1, Training loss 290.1488, Validation loss 324.3441
Epoch 1000, Training loss nan, Validation loss nan
Epoch 2000, Training loss nan, Validation loss nan
Epoch 3000, Training loss nan, Validation loss nan

Parameter containing:
tensor([[nan]], requires_grad=True) Parameter containing:
tensor([nan], requires_grad=True)


## Sequential Model 선언

In [18]:
seq_model = nn.Sequential(
    nn.Linear(1, 13),
    nn.Tanh(),
    nn.Linear(13, 1)
)

seq_model

Sequential(
  (0): Linear(in_features=1, out_features=13, bias=True)
  (1): Tanh()
  (2): Linear(in_features=13, out_features=1, bias=True)
)

In [19]:
[param.shape for param in seq_model.parameters()]

[torch.Size([13, 1]), torch.Size([13]), torch.Size([1, 13]), torch.Size([1])]

In [20]:
#출력 결과 확인하기 쉬움

for param in seq_model.named_parameters():
    print(param)

('0.weight', Parameter containing:
tensor([[ 0.6561],
        [-0.6070],
        [ 0.0408],
        [ 0.0965],
        [-0.1624],
        [-0.5406],
        [-0.0690],
        [-0.8924],
        [ 0.9006],
        [ 0.4805],
        [-0.5573],
        [ 0.3354],
        [ 0.9778]], requires_grad=True))
('0.bias', Parameter containing:
tensor([ 0.1394, -0.4071, -0.0745,  0.0953, -0.8816, -0.2687, -0.7986,  0.8354,
        -0.0917, -0.4309,  0.7145, -0.9079, -0.7487], requires_grad=True))
('2.weight', Parameter containing:
tensor([[-0.1399, -0.2245,  0.2579,  0.2149, -0.0188,  0.1534,  0.0946, -0.2034,
         -0.2162, -0.2158, -0.1338,  0.0037,  0.0343]], requires_grad=True))
('2.bias', Parameter containing:
tensor([0.0240], requires_grad=True))


In [27]:
seq_model = nn.Sequential(
    nn.Linear(1, 8),
    nn.Tanh(),
    nn.Linear(8, 1)
)


t_c_train = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0]).unsqueeze(1) 
t_c_val = torch.tensor([6.0, 13.0, 21.0]).unsqueeze(1)

t_u_train = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8]).unsqueeze(1) 
t_u_val = torch.tensor([48.4, 60.4, 68.4]).unsqueeze(1)


optimizer = torch.optim.SGD(seq_model.parameters(), lr = 1e-3)

training_loop(
    n_epochs=3000,
    optimizer = optimizer,
    model = linear_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val,
    t_c_train = t_c_train,
    t_c_val = t_c_val
)

RuntimeError: The size of tensor a (6) must match the size of tensor b (8) at non-singleton dimension 0